In [52]:
import tensorflow as tf
import torch as pt
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
tf.version.VERSION

'2.6.0'

In [2]:
img_path = './ISIC_Images'
mask_path = './ISIC_Masks'
img_width = 511
img_height = 384

In [3]:
images = tf.keras.utils.image_dataset_from_directory(img_path,image_size=(img_height, img_width))
masks = tf.keras.utils.image_dataset_from_directory(mask_path,image_size=(img_height, img_width))

Found 2594 files belonging to 1 classes.
Found 2594 files belonging to 1 classes.


In [4]:
test = load_img(f"{mask_path}/ISIC2018_Task1_Training_GroundTruth_x2/ISIC_0000000_segmentation.png", color_mode="grayscale")
test = img_to_array(test)
test /= 255

In [55]:
def correct_yolo_box(img_num):
    filepath = f"{mask_path}/ISIC2018_Task1_Training_GroundTruth_x2/ISIC_{img_num:07d}_segmentation.png"
    mask = load_img(filepath, color_mode = "grayscale")
    mask = img_to_array(mask)
    mask /= 255
    mask = np.squeeze(mask)
    img_height, img_width = mask.shape
    min_x = None
    max_x = None
    max_y = None
    min_y = None
    for i in range(0, img_height):
        if 1 in mask[i]:
            min_y = i
            break
    for i in range(img_height - 1, -1, -1):
        if 1 in mask[i]:
            max_y = i
            break
    for i in range(0, img_width):
        if 1 in mask.T[i]:
            min_x = i
            break
    for i in range(img_width - 1, -1, -1):
        if 1 in mask.T[i]:
            max_x = i
            break
    if min_y == None or max_y == None or min_x == None or max_x == None:
        print("Image is not a valid mask")
        raise
    width = max_x - min_x + 1
    height = max_y - min_y + 1
    centre_x = (min_x + max_x) / 2
    centre_y = (min_y + max_y) / 2
    return [width, height, centre_x, centre_y]

In [58]:
print(correct_yolo_box(20))

[318, 472, 308.5, 310.5]


In [59]:
def load_image_pixels(img_num):
    filepath = f"{img_path}/ISIC2018_Task1-2_Training_Input_x2/ISIC_{img_num:07d}.jpg"
    image = load_img(filepath)
    image = img_to_array(image)
    image = image.astype('float32')
    image /= 255
    return image

In [60]:
print(load_image_pixels(11))

[[[0.7607843  0.7607843  0.7607843 ]
  [0.39215687 0.39215687 0.39215687]
  [0.47843137 0.47843137 0.47843137]
  ...
  [0.47843137 0.47843137 0.47843137]
  [0.39607844 0.39607844 0.39607844]
  [0.76862746 0.76862746 0.76862746]]

 [[0.38431373 0.38431373 0.38431373]
  [0.         0.         0.        ]
  [0.03529412 0.03529412 0.03529412]
  ...
  [0.02745098 0.02745098 0.02745098]
  [0.00392157 0.00392157 0.00392157]
  [0.4509804  0.4509804  0.4509804 ]]

 [[0.46666667 0.46666667 0.46666667]
  [0.03529412 0.03529412 0.03529412]
  [0.05098039 0.05098039 0.05098039]
  ...
  [0.02352941 0.02352941 0.02352941]
  [0.01176471 0.01176471 0.01176471]
  [0.49803922 0.49803922 0.49803922]]

 ...

 [[0.4862745  0.4862745  0.4862745 ]
  [0.0627451  0.0627451  0.0627451 ]
  [0.09411765 0.09411765 0.09411765]
  ...
  [0.03137255 0.03137255 0.03137255]
  [0.00392157 0.00392157 0.00392157]
  [0.49019608 0.49019608 0.49019608]]

 [[0.3764706  0.3764706  0.3764706 ]
  [0.         0.         0.        ]
